In [1]:
%%capture
import stata_setup, os
if os.name == 'nt':
    stata_setup.config('C:/Program Files/Stata17/','mp')
else:
    stata_setup.config('/usr/local/stata17','mp')

In [2]:
%%stata -qui

use "../data/data", clear
rename log_flesch_kincaid_grade_level FKG
quietly tabulate year, generate(y_)
quietly tabulate cluster, generate(c_)

local journals  ecm jpe qje res  //AER based category

local jel_imp a_imp b_imp c_imp  e_imp f_imp g_imp h_imp i_imp j_imp k_imp /// 
		l_imp m_imp n_imp o_imp p_imp q_imp r_imp y_imp z_imp // D JEL based case

## Ridge Regression

The ridge regression estimator

$$
\widehat{\beta}_{\text {ridge }}=\left(\boldsymbol{X}^{\prime} \boldsymbol{X}+\lambda \boldsymbol{I}_{p}\right)^{-1} \boldsymbol{X}^{\prime} \boldsymbol{Y}
$$

where $\lambda>0$ is called the _ridge_ parameter.

In [3]:
%%stata -qui -eret steret
#delimit ;
quietly elasticnet linear FKG log_num_authors log_num_pages both_genders prop_women
        `journals' `jel_imp' y_2-y_20  c_2-c_215  jel_flag, alpha(0) lambda(5.34) nolog;
#delimit cr

In [4]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
steret['e(b)']

array([[-0.001,  0.002, -0.001, -0.003,  0.005, -0.003, -0.001,  0.002,
        -0.005, -0.003,  0.004, -0.001, -0.001, -0.001, -0.000,  0.000,
        -0.002, -0.003, -0.003, -0.004, -0.000,  0.000,  0.005,  0.003,
        -0.001,  0.004, -0.004,  0.003,  0.000, -0.001, -0.002, -0.002,
        -0.001, -0.002, -0.001, -0.001, -0.003,  0.001,  0.003, -0.000,
         0.002, -0.001,  0.002,  0.003, -0.001, -0.002,  0.026,  0.052,
         0.015,  0.015,  0.002,  0.007, -0.001,  0.000,  0.002,  0.039,
         0.050, -0.003,  0.002,  0.034, -0.014, -0.031, -0.025, -0.012,
         0.010, -0.014, -0.016, -0.001,  0.024, -0.007,  0.009,  0.002,
         0.026, -0.015, -0.005, -0.026,  0.027, -0.022, -0.056, -0.002,
        -0.021,  0.020,  0.007,  0.005,  0.037,  0.006, -0.004, -0.015,
         0.009, -0.008, -0.010,  0.009, -0.056, -0.004, -0.026, -0.013,
        -0.020, -0.001, -0.031, -0.025, -0.003, -0.004,  0.025, -0.006,
         0.000,  0.030,  0.024,  0.012,  0.043,  0.005, -0.000, 

In [ ]:
%%stata
#delimit ;
elasticnet linear FKG log_num_authors log_num_pages both_genders prop_women
        `journals' `jel_imp' y_2-y_20  c_2-c_215  jel_flag,
        alpha(0) lambda(1.3(.1)1.5) folds(4988) nolog;
#delimit cr


. #delimit ;
delimiter now ;
. elasticnet linear FKG log_num_authors log_num_pages both_genders prop_women
>         `journals' `jel_imp' y_2-y_20  c_2-c_215  jel_flag,
>         alpha(0) lambda(1.3(.1)1.5) folds(4988) nolog;


In [ ]:
%%stata -qui
#delimit ;
elasticnet linear FKG log_num_authors log_num_pages both_genders prop_women
        `journals' `jel_imp' y_2-y_20  c_2-c_215  jel_flag,
        alpha(0) lambda(1.4) nolog;
#delimit cr

In [ ]:
%stata matrix list r(table)

In [ ]:
%%stata
predict FKG_hat, xb
gen ehat = FKG - FKG_hat
set scheme s1mono
histogram ehat